**Table 1**
---

In [5]:
import pandas as pd
import numpy as np

# Function to format continuous variables
def format_continuous(grp_data):
    grp_data = grp_data[grp_data != -1]  # Exclude invalids
    mean = grp_data.mean()
    std = grp_data.std()
    median = grp_data.median()
    q1 = grp_data.quantile(0.25)
    q3 = grp_data.quantile(0.75)
    return f"{mean:.1f} ({std:.1f}); {median:.1f} [{q1:.1f}-{q3:.1f}]"

# Table 1 builder with group summaries
def summarize_table1_with_n(df, groupby_col, continuous, categorical):
    df = df.copy()
    groups = df[groupby_col].dropna().unique()
    group_sizes = df[groupby_col].value_counts().to_dict()

    table = {}

    # Continuous variables
    for col in continuous:
        row = {}
        for grp in groups:
            grp_data = df[df[groupby_col] == grp][col]
            row[grp] = format_continuous(grp_data)
        table[col] = row

    # Categorical variables
    for col in categorical:
        cats = df[col].dropna().unique()
        for cat in cats:
            row = {}
            for grp in groups:
                subset = df[df[groupby_col] == grp]
                count = (subset[col] == cat).sum()
                total = subset[col].isin([cat, 0, 1]).sum()  # valid total
                pct = (count / total * 100) if total else 0
                row[grp] = f"{count} ({pct:.1f}%)"
            table[f"{col} = {cat}"] = row

    df_out = pd.DataFrame.from_dict(table, orient='index')

    # Rename column headers with N counts
    group_names = {0: "TB Negative", 1: "TB Positive"}
    df_out.columns = [f"{group_names.get(grp, grp)} (N={group_sizes.get(grp, 0)})" for grp in df_out.columns]

    return df_out

# Load data
df_main = pd.read_csv('/Users/tylerstepaniak/Desktop/Jupyter Area/capstone-project/df_main_normalized.csv')

# Add placeholder column with NaN values
df_main["Sex"] = np.nan
df_main["Semiquant Xpert result"] = np.nan

# Generate Table 1
table1_df = summarize_table1_with_n(df_main, 'TB Status', ['Age', 'BMI'], ['HIV Status', 'Sex'])

# Rename row labels
table1_df = table1_df.rename(index={
    'Age': 'Age (years)',
    'BMI': 'Body Mass Index (kg/m²)',
    'HIV Status = 1': 'HIV Positive',
    'HIV Status = 0': 'HIV Negative',
    'HIV Status = -1': 'HIV Status Unknown',
    'Sex = 1': 'Male',
    'Sex = 0': 'Female'
})

# Add Summary Statistic column
summary_labels = {
    'Age (years)': 'mean (SD); median [IQR]',
    'Body Mass Index (kg/m²)': 'mean (SD); median [IQR]',
    'HIV Positive': 'n (%)',
    'HIV Negative': 'n (%)',
    'HIV Status Unknown': 'n (%)',
    'Male': 'n (%)',
    'Female': 'n (%)'
}
summary_stat_col = [summary_labels.get(var, '') for var in table1_df.index]
table1_df["Summary Statistic"] = summary_stat_col

# Add Xpert breakdown for TB Positive only
tb_positive_df = df_main[df_main['TB Status'] == 1]
xpert_levels = ['Trace', 'Low', 'Medium', 'High']
xpert_summary = {}

# Get current column names for dynamic insertion
tb_pos_col = [col for col in table1_df.columns if "TB Positive" in col][0]
tb_neg_col = [col for col in table1_df.columns if "TB Negative" in col][0]

for level in xpert_levels:
    count = (tb_positive_df['Semiquant Xpert result'] == level).sum()
    total = tb_positive_df['Semiquant Xpert result'].notna().sum()
    pct = (count / total * 100) if total else 0
    xpert_summary[f"Xpert: {level}"] = {
        tb_pos_col: f"{count} ({pct:.1f}%)",
        tb_neg_col: '',
        'Summary Statistic': 'n (%) (TB+)'
    }

# Combine with table1
xpert_df = pd.DataFrame.from_dict(xpert_summary, orient='index')
table1_df = pd.concat([table1_df, xpert_df])

# Export to Excel
table1_df.to_excel("output_table1.xlsx")